In [ ]:
import os, json
from pathlib import Path
from pandas import DataFrame
from mpcontribs.client import Client
from unflatten import unflatten

In [ ]:
client = Client()

**Load raw data**

In [ ]:
name = "screening_inorganic_pv"
indir = Path("/Users/patrick/gitrepos/mp/mpcontribs-data/ThinFilmPV")
files = {
    "summary": "SUMMARY.json",
    "absorption": "ABSORPTION-CLIPPED.json",
    "dos": "DOS.json",
    "formulae": "FORMATTED-FORMULAE.json"
}
data = {}

for k, v in files.items():
    path = indir / v
    with path.open(mode="r") as f:
        data[k] = json.load(f)
        
for k, v in data.items():
    print(k, len(v))

**Prepare contributions**

In [ ]:
config = {
    "SLME_500_nm": {"path": "SLME.500nm", "unit": "%"},
    "SLME_1000_nm": {"path": "SLME.1000nm", "unit": "%"},
    "E_g": {"path": "ΔE.corrected", "unit": "eV"},
    "E_g_d": {"path": "ΔE.direct", "unit": "eV"},
    "E_g_da": {"path": "ΔE.dipole", "unit": "eV"},
    "m_e": {"path": "mᵉ", "unit": "mₑ"},
    "m_h": {"path": "mʰ", "unit": "mₑ"}
}
columns = {c["path"]: c["unit"] for c in config.values()}
contributions = []

for mp_id, d in data["summary"].items():
    print(mp_id)
    formula = data["formulae"][mp_id].replace("<sub>", "").replace("</sub>", "")
    contrib = {"project": name, "identifier": mp_id, "data": {"formula": formula}, "tables": []}
    cdata = {v["path"]: f'{d[k]} {v["unit"]}' for k, v in config.items()}
    contrib["data"] = unflatten(cdata)
        
    df = DataFrame(data=data["absorption"][mp_id])
    df.columns = ["hν [eV]", "α [cm⁻¹]"]
    df.set_index("hν [eV]", inplace=True)
    #df.columns.name = columns_name # legend name
    df.attrs["name"] = "absorption"  # -> used as title by default
    #df.attrs["title"] = f'{title_prefix} of {doping_type}-type {titles[nm]}'
#     df.attrs["labels"] = {
#         "value": f'{nm}({doping_type}) [{u}]',  # y-axis label
#         #"variable": columns_name # alternative for df.columns.name
#     }
    contrib["tables"].append(df)

#     # df = DataFrame(data=dos_data[mp_id])
#     # df.columns = ['E [eV]', 'DOS [eV⁻¹]']

    contributions.append(contrib)
    break
    
len(contributions)

**Submit contributions**

In [ ]:
client.delete_contributions(name)
client.init_columns(columns)
client.submit_contributions(contributions)